Check if azure-ai-ml is installed

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.29.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\Users\Tony\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Connect to my worksapce

In [18]:
#import    required    libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
#Enter    details    of    your    Azure    Machine    Learning    workspace
subscription_id = 'c6cdfd9c-3767-4be7-a343-0acd27ddefb9'
resource_group = 'rg-tshen-0917'
workspace = 'ml-ws-tshen-0917'
#connect to the workspace
ml_client    =    MLClient(DefaultAzureCredential(),    subscription_id,
resource_group,    workspace)
ml_client

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x00000199577A5400>,
         subscription_id=c6cdfd9c-3767-4be7-a343-0acd27ddefb9,
         resource_group_name=rg-tshen-0917,
         workspace_name=ml-ws-tshen-0917)

Create source code directory if it does not exist

In [19]:
# create a local directory
import os

train_src_dir = "./src"
os.makedirs(train_src_dir, exist_ok=True)

Next, you'll create the endpoint by running the following cell. This may take several minutes. While your endpoint is being created, you can read about [what are Azure Machine Learning endpoints](https://learn.microsoft.com/azure/machine-learning/concept-endpoints).

In [24]:
%%writefile ./src/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="Path to input data")
    parser.add_argument("--test_train_ratio", type=float, default=0.25)
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--learning_rate", type=float, default=0.1)
    parser.add_argument("--registered_model_name", type=str, help="Model name")
    args = parser.parse_args()

    mlflow.start_run()
    mlflow.sklearn.autolog()

    credit_df = pd.read_csv(args.data, header=1, index_col=0)
    train_df, test_df = train_test_split(credit_df, test_size=args.test_train_ratio)

    y_train = train_df.pop("default payment next month")
    X_train = train_df.values
    y_test = test_df.pop("default payment next month")
    X_test = test_df.values

    clf = GradientBoostingClassifier(n_estimators=args.n_estimators, learning_rate=args.learning_rate)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))

    mlflow.sklearn.log_model(sk_model=clf, registered_model_name=args.registered_model_name, artifact_path=args.registered_model_name)
    # mlflow.sklearn.save_model(sk_model=clf, path=os.path.join(args.registered_model_name, "trained_model"))

    mlflow.end_run()

if __name__ == "__main__":
    main()

Overwriting ./src/main.py


Download data

In [21]:
# download data locally
import os
import urllib.request
os.makedirs("./data", exist_ok=True)

urllib.request.urlretrieve(
    "https://azuremlexamples.blob.core.windows.net/datasets/credit_card/default_of_credit_card_clients.csv",
    "./data/credit_card.csv"
)

('./data/credit_card.csv', <http.client.HTTPMessage at 0x19959527a70>)

Start mlfow

In [12]:
import mlflow
mlflow.set_experiment("credit_defaults_local")

2025/09/17 16:20:48 INFO mlflow.tracking.fluent: Experiment with name 'credit_defaults_local' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///f:/app/dp-100/azure-ml-labs/Labs/01/mlruns/494632932508169188', creation_time=1758144048333, experiment_id='494632932508169188', last_update_time=1758144048333, lifecycle_stage='active', name='credit_defaults_local', tags={}>

Run main.py

In [25]:
!python ./src/main.py --data ./data/credit_card.csv --registered_model_name local_model

              precision    recall  f1-score   support

           0       0.84      0.94      0.89      5896
           1       0.64      0.35      0.45      1604

    accuracy                           0.82      7500
   macro avg       0.74      0.65      0.67      7500
weighted avg       0.80      0.82      0.80      7500



2025/09/18 09:46:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/18 09:47:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 09:47:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'local_model' already exists. Creating a new version of this model...
Created version '5' of model 'local_model'.


Run main.py again

In [ ]:
!python ./scr/main.py ./data/credit_card.csv --registered_model_name local_model --experiment_name credit_defaults_local

Write main2.py

In [29]:
%%writefile ./src/main2.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="Path to input data")
    parser.add_argument("--test_train_ratio", type=float, default=0.25)
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--learning_rate", type=float, default=0.1)
    parser.add_argument("--registered_model_name", type=str, help="Model name")
    parser.add_argument("--experiment_name", type=str, default="credit_defaults_local")
    args = parser.parse_args()

    # Set experiment name
    mlflow.set_experiment(args.experiment_name)

    with mlflow.start_run():
        mlflow.sklearn.autolog()

        credit_df = pd.read_csv(args.data, header=1, index_col=0)
        train_df, test_df = train_test_split(credit_df, test_size=args.test_train_ratio)

        y_train = train_df.pop("default payment next month")
        X_train = train_df.values
        y_test = test_df.pop("default payment next month")
        X_test = test_df.values

        clf = GradientBoostingClassifier(n_estimators=args.n_estimators, learning_rate=args.learning_rate)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        print(classification_report(y_test, y_pred))

        mlflow.sklearn.log_model(sk_model=clf, registered_model_name=args.registered_model_name, artifact_path=args.registered_model_name)
        # mlflow.sklearn.save_model(sk_model=clf, path=os.path.join(args.registered_model_name, "trained_model"))

if __name__ == "__main__":
    main()


Overwriting ./src/main2.py


Run main2.py

In [30]:
!python ./src/main2.py --data ./data/credit_card.csv --registered_model_name local_model --experiment_name credit_defaults_local

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      5807
           1       0.67      0.38      0.48      1693

    accuracy                           0.82      7500
   macro avg       0.75      0.66      0.68      7500
weighted avg       0.80      0.82      0.80      7500



2025/09/18 09:57:28 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/09/18 09:57:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/18 09:57:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'local_model' already exists. Creating a new version of this model...
Created version '7' of model 'local_model'.
